In [1]:
#read single parquet file
import boto3
client = boto3.client('s3')
import math
import io
import pyarrow.parquet as pq
import pandas as pd 

buffer = io.BytesIO()
s3 = boto3.resource('s3')
bucket = 'columbia-stb'
key = 'local_day_id=2018-07-01/media_platform_type_cd=D/part-00000-tid-1729007679317263494-33e32f4b-b22b-4c44-b833-83fd26563c9a-6781.c000.snappy.parquet'
s3_object = s3.Object(bucket, key)

s3_object.download_fileobj(buffer)
table = pq.read_table(buffer)
df = table.to_pandas()
print(df.shape)
df.head()

(45431, 37)


,di_account_num,time_zone,start_timestamp_local,end_timestamp_local,start_timestamp_utc,end_timestamp_utc,duration_seconds,unique_session_id,viewing_end_timestamp_local,viewing_end_timestamp_utc,...,program_longsynopsis,program_runtime,program_contentrating,program_adult,program_local,program_sportssubtitle,program_tagids_title_type,adm_zip,adm_customer_type,vodasset_provider_id
0,QqD+x9TqGcUnH/hcBvaGSLXCauhhs0awD1PlngvxUl8=,-5,2018-07-01 23:26:25,2018-07-01 23:31:06,2018-07-02 04:26:25,2018-07-02 04:31:06,281.0000000000,00:00:01:3E:02:76_D_20180702042625_83,2018-07-01 23:31:06,2018-07-02 04:31:06,...,Newlyweds answer questions about each other to...,30.0,"PG,TVPG",F,F,,"ITheme:Game Show & Contest,Genre:Game Show",61571,RESIDENTIAL,
1,tPUCiqLU341PYO17j/iJjVj15wJ5996DGOZt5hBuxPk=,-4,2018-07-01 21:46:50,2018-07-01 21:57:08,2018-07-02 01:46:50,2018-07-02 01:57:08,618.0000000000,00:00:01:39:E7:85_D_20180702014650_222,2018-07-01 21:57:08,2018-07-02 01:57:08,...,When the owner of a clothing company is beaten...,60.0,"14+,TV14",F,F,,"Genre:Crime drama,KidsTheme:older teens (ages ...",02767,RESIDENTIAL,
2,gxlzvOuWeB0UIR/j9oO/kNpExoqCtWrelTATAvcnGFE=,-5,2018-07-01 20:12:54,2018-07-01 20:42:07,2018-07-02 01:12:54,2018-07-02 01:42:07,1753.0000000000,00:00:02:A1:AC:0F_D_20180702011254_172,2018-07-01 20:42:07,2018-07-02 01:42:07,...,"From Red Bull Ring in Spielberg, Austria.",115.0,E,F,F,,"Genre:SportingEvent,ITheme:Motor Sports,Genre:...",60172,RESIDENTIAL,
3,cTgsL52RasMASjp/yWaMcEOPFj5S7ELt3cBzt6VkTwY=,-7,2018-07-01 22:16:48,2018-07-01 22:19:32,2018-07-02 05:16:48,2018-07-02 05:19:32,164.0000000000,00:00:06:4D:2E:F4_D_20180702051648_129,2018-07-01 22:19:32,2018-07-02 05:19:32,...,The arrival of a wounded visitor in John Dorie...,60.0,"18+,TVMA,18+",F,F,,"ITheme:Drama,Genre:Science fiction,Genre:Horro...",98466,RESIDENTIAL,
4,ImCZMvVaHrLucZcxJ80lOh2HEuucOtQDVqeKbcQ3L8w=,-4,2018-07-01 01:31:37,2018-07-01 01:34:50,2018-07-01 05:31:37,2018-07-01 05:34:50,193.0000000000,00:00:05:2E:92:7F_D_20180701053137_110,2018-07-01 01:34:50,2018-07-01 05:34:50,...,"Storm-chasers (Helen Hunt, Bill Paxton) finali...",114.0,"14+,TV14,PG13,G,PG",F,F,,"Setting:Barn,Setting:Family home,Character:Nav...",33308,RESIDENTIAL,


In [2]:
print(df.isna().sum())
# #print(df["listing_starttime"].loc[df["listing_starttime"].isna()==True])
# #print(df["program_contentrating"])
# df.dtypes

di_account_num                         0
time_zone                              0
start_timestamp_local                  0
end_timestamp_local                    0
start_timestamp_utc                    0
end_timestamp_utc                      0
duration_seconds                       0
unique_session_id                      0
viewing_end_timestamp_local            0
viewing_end_timestamp_utc              0
viewing_duration_seconds               0
program_viewing_duration_seconds       0
station_title                          0
station_language                       0
station_timezone                       0
station_tagids_title_type              0
channel_language                       0
company_displayname                    0
company_description                    0
listing_id                             0
listing_starttime                   2985
listing_endtime                     2985
listing_airingtype                     0
program_id                             0
program_title   

In [3]:
#df=df.dropna()
#df.shape
df= df.dropna(subset=['program_runtime'])
print(df['program_runtime'].isna().sum())

0


In [4]:
df1 = df[['di_account_num','start_timestamp_utc','end_timestamp_utc','viewing_duration_seconds','program_id','program_runtime','program_contentrating','program_tagids_title_type']]

## Function to Extract Strings Seperated with Commas

In [5]:
def find_topics(test):
    counter = 0
    counters = [0]
    topics = list()
    for i in range(len(test)):
        if test[i] !=',':
            counter += 1
        else:
            counter += 1
            if test[counter].isupper(): 
                counters.append(counter-1)
            if len(counters)<3:
                topic = test[0:counters[-1]]
                if not topic in topics:
                    topics.append(topic)
            else:
                topic = test[counters[-2]+1:counters[-1]]
                if not topic in topics:
                    topics.append(topic)
    return topics

## all_topics is a list with split up Genres, Ithemes etc.

In [6]:
all_topics = list()
for k in df1.index:
    try:
        topics = find_topics(df1.loc[k,'program_tagids_title_type'])
        for j in topics:
            if not j in all_topics:
                all_topics.append(j)
    except:
        pass

df1.loc[10,'program_tagids_title_type']

'KidsTheme:teens (ages 13-14),ITheme:Drama,Genre:Drama,Genre:Teens,ITheme:Teen,Genre:Comedy drama,ITheme:Comedy-Drama'

In [7]:
all_topics

['ITheme:Game Show & Contest',
 'Genre:Crime drama',
 'KidsTheme:older teens (ages 15+)',
 'ITheme:Crime',
 'Genre:Drama',
 'Genre:SportingEvent',
 'ITheme:Motor Sports',
 'Genre:Sports',
 'Genre:Auto Racing',
 'ITheme:Drama',
 'Genre:Science fiction',
 'Genre:Horror',
 'ITheme:Horror',
 'Setting:Barn',
 'Setting:Family home',
 'Character:Navigator',
 'Genre:Thriller',
 'Setting:Oklahoma',
 'Subject:One last job',
 'Subject:Storm',
 'ITheme:Thriller',
 'Theme:Rivalry',
 'Character:Professor',
 'Setting:Drive-in theater',
 'KidsTheme:teens (ages 13-14)',
 'TimePeriod:1990s',
 'Subject:Contest',
 'Subject:Inventions',
 'Tone:Fascinating',
 'Character:Student',
 'Genre:Action & Adventure',
 'Theme:Underdog',
 'Setting:Laboratory',
 'TimePeriod:1960s',
 'Theme:Quest',
 'Character:Rival',
 'Character:Meteorologist',
 'ITheme:Action & Adventure',
 'Subject:Dangerous assignment',
 'Setting:Bridge',
 'Tone:Frenetic',
 'Subject:Scientific experiments',
 'Tone:Suspenseful',
 'Genre:Entertainment

## Creating a list of All Dimensions in program_tagids column

In [8]:
firsts = list()
for i in all_topics:
    first = i.split(':')[0]
    if not first in firsts:
        firsts.append(first)
        
firsts

['ITheme',
 'Genre',
 'KidsTheme',
 'Setting',
 'Character',
 'Subject',
 'Theme',
 'TimePeriod',
 'Tone',
 'General',
 'Keyword',
 'Music',
 '']

## Get Unique Genres

In [47]:
genres = list()
for i in all_topics:
    if i.find('Genre:')>-1:
        genre = i.split(':')[1]
        if not genre in genres:
            genres.append(genre)
print(len(genres),"Genres")
print(genres[1:10])
GenresD = pd.DataFrame([genres])
GenresD.to_csv('Genre Dictionary New.csv')

131 Genres
['Drama', 'SportingEvent', 'Sports', 'Auto Racing', 'Science fiction', 'Horror', 'Thriller', 'Action & Adventure', 'Entertainment']


## Creating a dictionary of the Genres / Assigning Name of the columns

In [33]:
genres_dict = {}
genres_dict['Account ID'] = [0]
for i in genres:
    genres_dict[i] = [0]

genres_dict

{'2018 World Cup': [0],
 'Account ID': [0],
 'Action & Adventure': [0],
 'Action Sports': [0],
 'Advice': [0],
 'Animals': [0],
 'Animated': [0],
 'Anime': [0],
 'Anthology': [0],
 'Art': [0],
 'Auction': [0],
 'Auto': [0],
 'Auto Racing': [0],
 'Awards': [0],
 'Baseball': [0],
 'Basketball': [0],
 'Beach Volleyball': [0],
 'Biography': [0],
 'Bowling': [0],
 'Boxing': [0],
 'Business & Finance': [0],
 'Card Games': [0],
 'Cheerleading': [0],
 "Children's/Family Entertainment": [0],
 'Christmas': [0],
 'Classic Movies': [0],
 'Classic Sport Event': [0],
 'Collectibles': [0],
 'Comedy': [0],
 'Comedy drama': [0],
 'Computers': [0],
 'Concert': [0],
 'Consumer': [0],
 'Country Music': [0],
 'Courtroom': [0],
 'Crafts': [0],
 'Crime': [0],
 'Crime drama': [0],
 'Dark comedy': [0],
 'Debate': [0],
 'Docudrama': [0],
 'Documentary': [0],
 'Drama': [0],
 'Entertainment': [0],
 'Event': [0],
 'Fantasy': [0],
 'Fashion': [0],
 'Fencing': [0],
 'Fitness': [0],
 'Food': [0],
 'Football': [0],
 '

In [34]:
# Loop over the rows and get the genres, ithemes
genres_by_account = list()
for k in df1.index:
    genres_k = [0]*len(genres)
    try:
        Id = df1.loc[k,'di_account_num']
        topics = find_topics(df1.loc[k,'program_tagids_title_type'])
        for j in topics:
            if j.find('Genre:')>-1:
                genre = j.split(':')[1]
                genres_k[genres.index(genre)] = 1
                
        genres_by_account.append([Id] + genres_k) 
        
    except:
        pass

df_genres = pd.DataFrame(genres_by_account, columns = genres_dict.keys())

In [35]:
df_genres.head()

,Account ID,Crime drama,Drama,SportingEvent,Sports,Auto Racing,Science fiction,Horror,Thriller,Action & Adventure,...,Jazz Music,Softball,Science & Technology,Fencing,Beach Volleyball,Swimming,Fitness,Crafts,St. Patrick's Day,Tennis
0,QqD+x9TqGcUnH/hcBvaGSLXCauhhs0awD1PlngvxUl8=,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,tPUCiqLU341PYO17j/iJjVj15wJ5996DGOZt5hBuxPk=,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,gxlzvOuWeB0UIR/j9oO/kNpExoqCtWrelTATAvcnGFE=,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,cTgsL52RasMASjp/yWaMcEOPFj5S7ELt3cBzt6VkTwY=,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ImCZMvVaHrLucZcxJ80lOh2HEuucOtQDVqeKbcQ3L8w=,0,1,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [37]:
import re
df_genres=df_genres.rename(columns=lambda x: re.sub('^Genre ', '', x))
df_genres.head(

,Account ID,Crime drama,Drama,SportingEvent,Sports,Auto Racing,Science fiction,Horror,Thriller,Action & Adventure,...,Jazz Music,Softball,Science & Technology,Fencing,Beach Volleyball,Swimming,Fitness,Crafts,St. Patrick's Day,Tennis
0,QqD+x9TqGcUnH/hcBvaGSLXCauhhs0awD1PlngvxUl8=,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,tPUCiqLU341PYO17j/iJjVj15wJ5996DGOZt5hBuxPk=,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,gxlzvOuWeB0UIR/j9oO/kNpExoqCtWrelTATAvcnGFE=,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,cTgsL52RasMASjp/yWaMcEOPFj5S7ELt3cBzt6VkTwY=,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ImCZMvVaHrLucZcxJ80lOh2HEuucOtQDVqeKbcQ3L8w=,0,1,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [38]:
genre1=pd.read_csv('Genre Dictionary.csv',error_bad_lines=False)

genre1.head()

,Drama,Sports,Reality,Entertainment,Documentary,Kids,Family,Animals,News,Movie,Music,Teen,Science & Technology,Health & Fitness,Religion,Shopping,Miscellaneous
0,Action & Adventure,Sports talk,Legal,Action & Adventure,Nature,Cartoon & Animation,Children's/Family Entertainment,Pets,Nature,Thriller,Western,Comedy,Sci-Fi & Fantasy,Health,Fundraiser/Telethon,Consumer,Anthology
1,Fantasy,Basketball,Outdoors,Fantasy,Crime,Animated,Home & How-To,Wild animals,Real Estate,Military & War,Local,Education,Aviation,Fitness,Religion,Fashion,Special
2,Miniseries,Outdoors,Talk,Martial Arts,History & Biography,Comedy,Home & Garden,Ocean creatures,Crime,Legal,Musical,Romance,Environment,Health & Fitness,NaN,Thanksgiving,Independent
3,Crime,Martial Arts,Travel,Travel,Medical,Education,Animated,Horses and farm animals,Medical,Western,Hip-Hop & Rap Music,Teens,Science and nature,NaN,NaN,Valentine's Day,Variety
4,Thriller,Tennis,Food,Sitcom,Military & War,Holiday,Medical,Animals,Newscast,Horror,NaN,Mystery,Robots,NaN,NaN,Halloween,How-To


In [39]:
#Calling a Genre Dictionary
genre_dict={}
for i in genre1.columns.tolist():
    value= [x for x in list(genre1[str(i)])if str(x) != 'nan']
    genre_dict[i]=value
genre_dict

{'Animals': ['Pets',
  'Wild animals',
  'Ocean creatures',
  'Horses and farm animals',
  'Animals'],
 'Documentary': ['Nature',
  'Crime',
  'History & Biography',
  'Medical',
  'Military & War',
  'Outdoors',
  'History',
  'Biography',
  'Politics & Government',
  'Auto',
  'Self improvement',
  'Environment',
  'Dinosaurs',
  'Discovery',
  'Gay and Lesbian',
  'Documentary'],
 'Drama': ['Action & Adventure',
  'Fantasy',
  'Miniseries',
  'Crime',
  'Thriller',
  'Military & War',
  'Horror',
  'Local',
  'Mystery & Suspense',
  'Travel',
  'Sitcom',
  'Comedy',
  'Romance',
  'Comedy-Drama',
  'Mystery',
  'Historical drama',
  'Science fiction',
  'Crime drama',
  'Comedy drama',
  'Romantic comedy',
  'Musical comedy',
  'Soap Opera',
  'Advice',
  'Dark comedy',
  'Art expert',
  'Theater',
  'not for kids',
  'Adventures',
  'Adventure',
  'Escape',
  'Love',
  'Rescue',
  'Pursuit',
  'Revenge',
  'Growing up',
  'Sacrifice',
  'Temptation',
  'Gay and Lesbian',
  'Drama']

## Creating Dummy Columns For each Genre Grouping

In [40]:
dftemp=df1
for key,value in genre_dict.items():
    g_list=value
    for g in g_list:
        dftemp[str(key)]=dftemp['program_tagids_title_type'].apply(lambda x:1 if str(x).find(g)>-1 else 0)
dftemp.head()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,di_account_num,start_timestamp_utc,end_timestamp_utc,viewing_duration_seconds,program_id,program_runtime,program_contentrating,program_tagids_title_type,Drama,Sports,...,Animals,News,Movie,Music,Teen,Science & Technology,Health & Fitness,Religion,Shopping,Miscellaneous
0,QqD+x9TqGcUnH/hcBvaGSLXCauhhs0awD1PlngvxUl8=,2018-07-02 04:26:25,2018-07-02 04:31:06,281.0000000000,8577965228089996112,30.0,"PG,TVPG","ITheme:Game Show & Contest,Genre:Game Show",0,0,...,0,0,0,0,0,0,0,0,0,0
1,tPUCiqLU341PYO17j/iJjVj15wJ5996DGOZt5hBuxPk=,2018-07-02 01:46:50,2018-07-02 01:57:08,618.0000000000,7577879569959863112,60.0,"14+,TV14","Genre:Crime drama,KidsTheme:older teens (ages ...",1,0,...,0,0,0,0,0,0,0,0,0,0
2,gxlzvOuWeB0UIR/j9oO/kNpExoqCtWrelTATAvcnGFE=,2018-07-02 01:12:54,2018-07-02 01:42:07,1753.0000000000,8892192327857233112,115.0,E,"Genre:SportingEvent,ITheme:Motor Sports,Genre:...",0,1,...,0,0,0,0,0,0,0,0,0,0
3,cTgsL52RasMASjp/yWaMcEOPFj5S7ELt3cBzt6VkTwY=,2018-07-02 05:16:48,2018-07-02 05:19:32,164.0000000000,6163066522459043112,60.0,"18+,TVMA,18+","ITheme:Drama,Genre:Science fiction,Genre:Horro...",1,0,...,0,0,0,0,0,0,0,0,0,0
4,ImCZMvVaHrLucZcxJ80lOh2HEuucOtQDVqeKbcQ3L8w=,2018-07-01 05:31:37,2018-07-01 05:34:50,193.0000000000,5293156272934801112,114.0,"14+,TV14,PG13,G,PG","Setting:Barn,Setting:Family home,Character:Nav...",1,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
dftemp['Drama_duration']=dftemp['viewing_duration_seconds']*dftemp['Drama']
dftemp['Sports_duration']=dftemp['viewing_duration_seconds']*dftemp['Sports']
dftemp['Reality_duration']=dftemp['viewing_duration_seconds']*dftemp['Reality']
dftemp['Entertainment_duration']=dftemp['viewing_duration_seconds']*dftemp['Entertainment']
dftemp['Documentary_duration']=dftemp['viewing_duration_seconds']*dftemp['Documentary']
dftemp['Kids_duration']=dftemp['viewing_duration_seconds']*dftemp['Kids']
dftemp['Family_duration']=dftemp['viewing_duration_seconds']*dftemp['Family']
dftemp['Animals_duration']=dftemp['viewing_duration_seconds']*dftemp['Animals']
dftemp['News_duration']=dftemp['viewing_duration_seconds']*dftemp['News']
dftemp['Movie_duration']=dftemp['viewing_duration_seconds']*dftemp['Movie']
dftemp['Music_duration']=dftemp['viewing_duration_seconds']*dftemp['Music']
dftemp['Teen_duration']=dftemp['viewing_duration_seconds']*dftemp['Teen']
dftemp['Science & Technology_duration']=dftemp['viewing_duration_seconds']*dftemp['Science & Technology']
dftemp['Health & Fitness_duration']=dftemp['viewing_duration_seconds']*dftemp['Health & Fitness']
dftemp['Religion_duration']=dftemp['viewing_duration_seconds']*dftemp['Religion']
dftemp['Shopping_duration']=dftemp['viewing_duration_seconds']*dftemp['Shopping']
dftemp['Miscellaneous_duration']=dftemp['viewing_duration_seconds']*dftemp['Miscellaneous']
dftemp

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexi

,di_account_num,start_timestamp_utc,end_timestamp_utc,viewing_duration_seconds,program_id,program_runtime,program_contentrating,program_tagids_title_type,Drama,Sports,...,Animals_duration,News_duration,Movie_duration,Music_duration,Teen_duration,Science & Technology_duration,Health & Fitness_duration,Religion_duration,Shopping_duration,Miscellaneous_duration
0,QqD+x9TqGcUnH/hcBvaGSLXCauhhs0awD1PlngvxUl8=,2018-07-02 04:26:25,2018-07-02 04:31:06,281.0000000000,8577965228089996112,30.0,"PG,TVPG","ITheme:Game Show & Contest,Genre:Game Show",0,0,...,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10
1,tPUCiqLU341PYO17j/iJjVj15wJ5996DGOZt5hBuxPk=,2018-07-02 01:46:50,2018-07-02 01:57:08,618.0000000000,7577879569959863112,60.0,"14+,TV14","Genre:Crime drama,KidsTheme:older teens (ages ...",1,0,...,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10
2,gxlzvOuWeB0UIR/j9oO/kNpExoqCtWrelTATAvcnGFE=,2018-07-02 01:12:54,2018-07-02 01:42:07,1753.0000000000,8892192327857233112,115.0,E,"Genre:SportingEvent,ITheme:Motor Sports,Genre:...",0,1,...,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10
3,cTgsL52RasMASjp/yWaMcEOPFj5S7ELt3cBzt6VkTwY=,2018-07-02 05:16:48,2018-07-02 05:19:32,164.0000000000,6163066522459043112,60.0,"18+,TVMA,18+","ITheme:Drama,Genre:Science fiction,Genre:Horro...",1,0,...,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10
4,ImCZMvVaHrLucZcxJ80lOh2HEuucOtQDVqeKbcQ3L8w=,2018-07-01 05:31:37,2018-07-01 05:34:50,193.0000000000,5293156272934801112,114.0,"14+,TV14,PG13,G,PG","Setting:Barn,Setting:Family home,Character:Nav...",1,0,...,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10
5,MHraetBu6LHZshYarbQc1C9kQ9RNd5bYgvApEt0kHdc=,2018-07-01 11:57:29,2018-07-01 12:19:40,1331.0000000000,7097447008608553112,60.0,"14+,TV14","KidsTheme:teens (ages 13-14),Genre:Drama,IThem...",1,0,...,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10
6,+ZEiskTrEOQEbsla65YBgqZYM4apTJbdwFRX/gPvbPE=,2018-07-02 01:01:20,2018-07-02 01:03:47,147.0000000000,7813245655137546112,60.0,"PG,TVPG","Genre:Reality,KidsTheme:big kids (ages 8-9),IT...",0,0,...,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10
7,uqvcVaX2tq8DrCKD3Swsme3+UElvZSX09oeQh43eNsM=,2018-07-02 02:37:06,2018-07-02 02:37:57,51.0000000000,6821803865956142112,30.0,"PG,TVPG","ITheme:Game Show & Contest,Genre:Game Show",0,0,...,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10
8,+g+UbXOaY5MVPCa9u1BanYL3Bgop+2+hVNXWIUNkGdE=,2018-07-02 03:07:53,2018-07-02 03:13:44,351.0000000000,5931862417738374112,60.0,"18+,TVMA,18+","ITheme:Drama,Genre:Comedy,KidsTheme:older teen...",1,0,...,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10
9,L+9/QpVCfQBVfmpXAHdnsy03VvFhNnqh9mRupjGb0+8=,2018-07-01 15:49:52,2018-07-01 15:56:32,400.0000000000,8942899334237468112,60.0,"G,TVG,PG,G","Genre:Drama,Genre:Home improvement,ITheme:Dram...",1,0,...,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10,0E-10


In [ ]:
df['prog_conc_3']=df['prog_conc_3'].fillna(df['prog_conc_2'])
df.dropna('program_viewing_duration_seconds')
df1=df.loc[df['prog_conc_1'].notnull()]
df1.isna().sum()

we will not be using timestamps and program tagids type as they are already separated in columns, program_viewing_duration_seconds as it is the same as duration_seconds and unique id columns like account number and unique id

In [ ]:
# filling company displayname, runtime, and genres
df1['company_displayname']=df1['company_displayname'].fillna('Unknown')

#filling runtimes with mean values
df1['program_runtime']=df1['program_runtime'].fillna(df['program_runtime'].mean())
